In [3]:
%matplotlib inline
import bokeh
from bokeh.io import output_notebook
from bokeh.charts import Scatter, output_file, show, Bar
from bokeh.palettes import Accent, Category20c
from bokeh.models import Range1d
from bokeh.resources import CDN
from bokeh.embed import file_html, autoload_static

import os
import sys
from itertools import combinations
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats.mstats import gmean
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
import pickle

from notebooks import log_progress

sns.set_style('whitegrid')

# Dataset Clustering

Cluster exploration of our [GTEx](https://www.gtexportal.org/home/) and [TCGA](https://cancergenome.nih.gov/) dataset of [RNA-seq samples](https://xenabrowser.net/datapages/?host=https://toil.xenahubs.net) to help grasp the complexity and "structure" of the data. 

All rendered plots are available:  

## Cluster Entire Dataset Using t-SNE and PCA

Voices in the wind have mentioned that running PCA before t-SNE is a common preprocessing step that drastically reduces how long it takes to render a final t-SNE plot. "Components" refers to the number of dimensions for each sample, which starts at ~20,000 protein-coding genes. 

- Read in dataframes across all tissues
- Run PCA to reduce from ~20k to 10 components
- Run t-SNE to get 2 components
- Plot, make interactive

Read in expression dataframes for each tissue

In [5]:
exp_dir = '../../data/expression-PC/'
tissues = os.listdir(exp_dir)
tsv = 'combined-gtex-tcga-counts-protein-coding.tsv'
exp_df = pd.concat([pd.read_csv(os.path.join(exp_dir, t, tsv), 
                             sep='\t', index_col=0) for t in tissues], axis=1)

Remove duplicate columns

In [6]:
exp_df = exp_df.T.groupby(level=0).first().T

Subset for just tumor and normal samples

In [7]:
samples = [x for x in exp_df.columns if x.startswith('GTEX') or (x.endswith('01') or x.endswith('11'))]
exp_df = exp_df[samples]

Transpose dataframe (for PCA)

In [8]:
exp_df = exp_df.T

Apply $\mbox{log2}(x + 1)$ normalization to counts.

In [9]:
exp_df = exp_df.apply(lambda x: np.log2(x + 1))

Reduce number of features down to 50 with TruncatedSVD to speed up t-SNE

In [10]:
y = TruncatedSVD(n_components=50, random_state=0).fit_transform(np.array(exp_df))

Run t-SNE

In [11]:
for perplexity in [20, 30, 40, 50]:
    for lr in [500, 750, 1000]:      
        model = TSNE(n_components=2, random_state=1, perplexity=50, learning_rate=1000, verbose=2)
        z = model.fit_transform(np.array(y))
        
        tissue_map = pickle.load(open('../../data/tissue_map.pickle', 'rb'))
        df = pd.DataFrame()
        df['sample'] = samples
        df['tissue'] = [tissue_map[x].capitalize() for x in samples]
        df['x'] = z[:, 0]
        df['y'] = z[:, 1]

        types = []
        for sample in samples:
            if sample.startswith('GTEX'):
                types.append('GTEX')
            elif sample.endswith('01'):
                types.append('TCGA-Tumor')
            elif sample.endswith('11'):
                types.append('TCGA-Normal')
        df['type'] = types
        
        tooltips=[
        ('Tissue', '@tissue'),
        ('Type', '@type'),
        ('Sample', '@sample'),
        ]

    p = Scatter(df, x='x', y='y', title="t-SNE of GTEx and TCGA RNA-seq Expression",
                xlabel="1", ylabel="2",
                color='tissue',
                tooltips=tooltips,
                legend=True,
                plot_width=1024, plot_height=1024,
                palette=Category20c[20],
                responsive=True)

    p.title.align = 'center'

    js, tag = autoload_static(p, CDN, "js/bokeh/expression-PC.js")
    with open("expression-PC.js", 'w') as f:
        f.write(js)
    with open("expression-PC.tag", 'a') as f:
        f.write(tag)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 10864
[t-SNE] Computed conditional probabilities for sample 2000 / 10864
[t-SNE] Computed conditional probabilities for sample 3000 / 10864
[t-SNE] Computed conditional probabilities for sample 4000 / 10864
[t-SNE] Computed conditional probabilities for sample 5000 / 10864
[t-SNE] Computed conditional probabilities for sample 6000 / 10864
[t-SNE] Computed conditional probabilities for sample 7000 / 10864
[t-SNE] Computed conditional probabilities for sample 8000 / 10864
[t-SNE] Computed conditional probabilities for sample 9000 / 10864
[t-SNE] Computed conditional probabilities for sample 10000 / 10864
[t-SNE] Computed conditional probabilities for sample 10864 / 10864
[t-SNE] Mean sigma: 29.924147
[t-SNE] Iteration 25: error = 1.1560246, gradient norm = 0.0005223
[t-SNE] Iteration 25: gradient norm 0.000522. Finished.
[t-SNE] Iteration 50: err

Create dataframe object which we'll use to render the plot

Plot

## Clustering for All Individual Tissues

Let's functionize the above to stay DRY and repeat for each individual tissue, labeling the dataset each tissue comes from as the primary legend. 

In [24]:
def process_and_plot_tissue(df, title):
    # Remove duplicate columns
    df = df.T.groupby(level=0).first().T
    # Subset just tumor / normal
    samples = [x for x in df.columns if x.startswith('GTEX') or (x.endswith('01') or x.endswith('11'))]
    df = df[samples]
    # Transpose DF for PCA / t-SNE
    df = df.T
    # Log Normalize
    df = df.apply(lambda x: np.log2(x + 1))
    # Reduce to 50 components
    y = TruncatedSVD(n_components=50, random_state=0).fit_transform(np.array(df))
    # t-SNE
    model = TSNE(n_components=2, random_state=1)
    z = model.fit_transform(np.array(y))
    
    # Create dataframe for plot
    tissue_map = pickle.load(open('../../data/tissue_map.pickle', 'rb'))
    df = pd.DataFrame()
    df['sample'] = samples
    df['tissue'] = [tissue_map[x].capitalize() for x in samples]
    df['x'] = z[:, 0]
    df['y'] = z[:, 1]
    
    # Get type of samples
    types = []
    for sample in samples:
        if sample.startswith('GTEX'):
            types.append('GTEX')
        elif sample.endswith('01'):
            types.append('TCGA-Tumor')
        elif sample.endswith('11'):
            types.append('TCGA-Normal')
    df['type'] = types
    
    # Specify tooltip
    tooltips=[
    ('Tissue', '@tissue'),
    ('Type', '@type'),
    ('Sample', '@sample'),
    ]
    
    # Plot
    p = Scatter(df, x='x', y='y', title="t-SNE of: " + title,
            xlabel="1", ylabel="2",
            color='type',
            tooltips=tooltips,
            legend=True,
            plot_width=400, plot_height=400,
            palette=Accent[3],
            active_drag="pan",
            active_scroll="wheel_zoom",
            responsive=True)

    js, tag = autoload_static(p, CDN, "js/bokeh/expression-PC-{}.js".format(title))
    with open("expression-PC-{}.js".format(title), 'w') as f:
        f.write(js)
    with open("tags".format(title), 'a') as f:
        f.write(tag)

In [25]:
for tissue in log_progress(tissues):
    df = pd.read_csv(os.path.join(exp_dir, tissue, tsv), sep='\t', index_col=0)
    process_and_plot_tissue(df, title=tissue)

# Clustering with ComBat

Nonparametric
Tissue as a covariate
Monte-Carlo fix


In [87]:
exp_dir = '../../data/expression-CBNT-PC/'
tissues = sorted(os.listdir(exp_dir))
tsv = 'expression-CBNT-PC.tsv'

In [86]:
for tissue in log_progress(tissues):
    df = pd.read_csv(os.path.join(exp_dir, tissue, tsv), sep='\t', index_col=0)
    process_and_plot_tissue(df, title=tissue + "-combat")

## Entire Dataset with ComBat

In [88]:
exp_df = pd.concat([pd.read_csv(os.path.join(exp_dir, t, tsv), sep='\t', index_col=0) for t in tissues], axis=1)
exp_df = exp_df.T.groupby(level=0).first().T
samples = [x for x in exp_df.columns if x.startswith('GTEX') or (x.endswith('01') or x.endswith('11'))]
exp_df = exp_df[samples]
exp_df = exp_df.T
exp_df = exp_df.apply(lambda x: np.log2(x + 1))
print 'Running TruncatedSVD'
y = TruncatedSVD(n_components=50, random_state=0).fit_transform(np.array(exp_df))
model = TSNE(n_components=2, random_state=1, perplexity=50, learning_rate=1000, verbose=2)
z = model.fit_transform(np.array(y))

tissue_map = pickle.load(open('../../data/tissue_map.pickle', 'rb'))
df = pd.DataFrame()
df['sample'] = samples
df['tissue'] = [tissue_map[x].capitalize() for x in samples]
df['x'] = z[:, 0]
df['y'] = z[:, 1]

types = []
for sample in samples:
    if sample.startswith('GTEX'):
        types.append('GTEX')
    elif sample.endswith('01'):
        types.append('TCGA-Tumor')
    elif sample.endswith('11'):
        types.append('TCGA-Normal')
df['type'] = types

tooltips=[
    ('Tissue', '@tissue'),
    ('Type', '@type'),
    ('Sample', '@sample'),
]

p = Scatter(df, x='x', y='y', title="t-SNE of GTEx and TCGA RNA-seq Expression",
            xlabel="1", ylabel="2",
            color='tissue',
            tooltips=tooltips,
            legend=True,
            plot_width=1024, plot_height=1024,
            palette=Category20c[20])

output_file("expression-PC-combat.html")
js, tag = autoload_static(p, CDN, "js/bokeh/expression-PC.js")
with open("expression-PC-combat.js", 'w') as f:
    f.write(js)
with open("expression-PC-combat.tag", 'a') as f:
    f.write(tag)
show(p)

Running TruncatedSVD
[t-SNE] Computing pairwise distances...
[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 10864
[t-SNE] Computed conditional probabilities for sample 2000 / 10864
[t-SNE] Computed conditional probabilities for sample 3000 / 10864
[t-SNE] Computed conditional probabilities for sample 4000 / 10864
[t-SNE] Computed conditional probabilities for sample 5000 / 10864
[t-SNE] Computed conditional probabilities for sample 6000 / 10864
[t-SNE] Computed conditional probabilities for sample 7000 / 10864
[t-SNE] Computed conditional probabilities for sample 8000 / 10864
[t-SNE] Computed conditional probabilities for sample 9000 / 10864
[t-SNE] Computed conditional probabilities for sample 10000 / 10864
[t-SNE] Computed conditional probabilities for sample 10864 / 10864
[t-SNE] Mean sigma: 26.242107
[t-SNE] Iteration 25: error = 1.1507380, gradient norm = 0.0006062
[t-SNE] Iteration 25: gradient norm 0.000606. Finished.
[t-S

# Sample Counts for Datasets

In [2]:
tissues = ['adrenal', 'bladder', 'breast', 'cervix', 'colon', 'esophagus', 'kidney', 
           'liver', 'lung_adenocarcinoma', 'lung_mesothelioma', 'lung_squamous', 'ovary', 
           'pancreas', 'prostate', 'skin', 'head_and_neck', 'stomach', 'testis', 
           'thyroid', 'uterus_carcinosarcoma', 'uterus_endometrioid']
tcga_t = ['77', '407', '1092', '303', '287', '181', '530', '369', '513', '87', '498', 
          '420', '178', '494', '102', '1040', '413', '148', '504', '57', '180']
tcga_n = ['0', '19', '113', '3', '41', '13', '72', '50', '59', '0', '50', '0', '4', 
          '51', '1', '88', '36', '0', '59', '0', '23']
gtex = ['125', '9', '178', '10', '140', '379', '27', '110', '287', '287', '287', '88', 
        '165', '100', '556', '556', '173', '165', '278', '78', '78']

tc = pd.DataFrame()
tc['counts'] = [int(x) for x in gtex + tcga_t + tcga_n]
tc['dataset'] = ['gtex' for _ in xrange(len(gtex))] + \
                ['tumor' for _ in xrange(len(tcga_t))] + \
                ['normal' for _ in xrange(len(tcga_n))]
tc['tissue'] = tissues * 3

In [3]:
tooltips=[
    ('# Samples', '@height'),
]
b = Bar(tc, label='tissue', values='counts', group='dataset',
       plot_width=1024, plot_height=512, tooltips=tooltips,
       title='Number of Samples per Tissue Across Datasets',
       responsive=True)

b.title.align = 'center'

output_file("tissue-count.html")
show(b)

js, tag = autoload_static(b, CDN, "js/bokeh/tissue-counts.js")
with open("tissue-counts.js", 'w') as f:
    f.write(js)
with open("tissue-counts.tag", 'a') as f:    f.write(tag)

# Quantile Normalization pre-Reduction
Let's skip log normalization and use a dataset that's been quantile-normalized to see how it affects clustering.


In [4]:
exp_df = pd.read_csv('../../data/expression-PC-QN/expression-PC-QN.tsv', index_col=0, sep='\t')

In [11]:
exp_df.shape

(19797, 10864)

Transpose so genes are the columns (features)

In [12]:
exp_df = exp_df.T

In [13]:
y = TruncatedSVD(n_components=50, random_state=0).fit_transform(np.array(exp_df))

In [14]:
model = TSNE(n_components=2, random_state=1, perplexity=50, learning_rate=1000, verbose=2)
z = model.fit_transform(np.array(y))

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 151 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 10864
[t-SNE] Computed conditional probabilities for sample 2000 / 10864
[t-SNE] Computed conditional probabilities for sample 3000 / 10864
[t-SNE] Computed conditional probabilities for sample 4000 / 10864
[t-SNE] Computed conditional probabilities for sample 5000 / 10864
[t-SNE] Computed conditional probabilities for sample 6000 / 10864
[t-SNE] Computed conditional probabilities for sample 7000 / 10864
[t-SNE] Computed conditional probabilities for sample 8000 / 10864
[t-SNE] Computed conditional probabilities for sample 9000 / 10864
[t-SNE] Computed conditional probabilities for sample 10000 / 10864
[t-SNE] Computed conditional probabilities for sample 10864 / 10864
[t-SNE] Mean sigma: 192041.286228
[t-SNE] Iteration 25: error = 1.1545317, gradient norm = 0.0005179
[t-SNE] Iteration 25: gradient norm 0.000518. Finished.
[t-SNE] Iteration 50:

In [17]:
samples = exp_df.index

In [18]:
tissue_map = pickle.load(open('../../data/tissue_map.pickle', 'rb'))
df = pd.DataFrame()
df['sample'] = samples
df['tissue'] = [tissue_map[x].capitalize() for x in samples]
df['x'] = z[:, 0]
df['y'] = z[:, 1]

types = []
for sample in samples:
    if sample.startswith('GTEX'):
        types.append('GTEX')
    elif sample.endswith('01'):
        types.append('TCGA-Tumor')
    elif sample.endswith('11'):
        types.append('TCGA-Normal')
df['type'] = types

In [19]:
tooltips=[
    ('Tissue', '@tissue'),
    ('Type', '@type'),
    ('Sample', '@sample'),
]

p = Scatter(df, x='x', y='y', title="t-SNE of GTEx and TCGA RNA-seq Expression",
            xlabel="1", ylabel="2",
            color='tissue',
            tooltips=tooltips,
            legend=True,
            plot_width=1024, plot_height=1024,
            palette=Category20c[20],
            responsive=True)

p.title.align = 'center'

show(p)

js, tag = autoload_static(p, CDN, "js/bokeh/expression-PC.js")
with open("expression-PC-QN.js", 'w') as f:
    f.write(js)
with open("expression-PC-QN.tag", 'a') as f:
    f.write(tag)

Unfortunately, it looks terrible. 

# Old
I initially looked at size factor rescaling to improve plots but tweaking t-SNE and combining with TruncatedSVD over PCA for initial component reduction worked better.

### Size Factor Rescaling

Size factors are calculated to render counts from different samples, which may have been sequenced to different depths, comparable. 

j...m = Samples <br>
i...n = Genes <br>
k = counts

One size factor is calculated per sample

$$\hat{s}_j = \mbox{median_i} \frac{k_{ij}}{\Big( \Pi^m_{v=1}k_{iv} \Big)^{1/m}}$$

We can then produce what the DESeq2 authors call "variance stabalized normalized counts" by computing:
$\mbox{log2}\big(\frac{\mbox{count}}{\mbox{size_factor}}\big)$ for all counts in the matrix

In [3]:
def size_factor_scale(df):
    # Calculate denominator
    # The geometric mean for all genes across all samples
    # We'll pad the counts with a +1 so the geometric mean is never 0
    df = df.apply(lambda x: x + 1)
    denom = gmean(df, axis=1)  # Length n
    temp = df.divide(denom, axis=0)
    size_factors = temp.median(axis=0)
    return df.divide(size_factors, axis=1).apply(lambda x: np.log2(x))

def quantile_normalize(df):
    """
    Quantile normalization is a technique for making two distributions identical in statistical properties.
    Assuming genes by samples
    from: http://stackoverflow.com/questions/37935920/quantile-normalization-on-pandas-dataframe
    
    Gene x Sample matrix
    """
    rank_mean = df.stack().groupby(df.rank(method='first').stack().astype(int)).mean()
    return df.rank(method='min').stack().astype(int).map(rank_mean).unstack()